# Libraries

In [1]:
# install.packages('xgboost')

Installing package into ‘/home/ggomarr/R/x86_64-pc-linux-gnu-library/3.3’
(as ‘lib’ is unspecified)


In [3]:
library(caTools)
library(ggplot2)
library(repr)
options(repr.plot.width=4, repr.plot.height=4)
library(xgboost)

# Without one hot encoding

## Read data

In [4]:
root_dir="~/Documents/Education/20170623 Udemy - Machine Learning A-Z: Hands-On Python and R in Data Science/"
work_dir="Course data/Part 10 - Model Selection & Boosting/Section 49 - XGBoost"
setwd(paste(root_dir,work_dir,sep=''))
df=read.csv('Churn_Modelling.csv')[c('Geography','Gender','Age','CreditScore','EstimatedSalary',
                                     'Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember',
                                     'Exited')]
str(df)

'data.frame':	10000 obs. of  11 variables:
 $ Geography      : Factor w/ 3 levels "France","Germany",..: 1 3 1 1 3 3 1 2 1 1 ...
 $ Gender         : Factor w/ 2 levels "Female","Male": 1 1 1 1 1 2 2 1 2 2 ...
 $ Age            : int  42 41 42 39 43 44 50 29 44 27 ...
 $ CreditScore    : int  619 608 502 699 850 645 822 376 501 684 ...
 $ EstimatedSalary: num  101349 112543 113932 93827 79084 ...
 $ Tenure         : int  2 1 8 1 2 8 7 4 4 2 ...
 $ Balance        : num  0 83808 159661 0 125511 ...
 $ NumOfProducts  : int  1 1 3 2 1 2 2 4 2 1 ...
 $ HasCrCard      : int  1 0 1 0 1 1 1 1 0 1 ...
 $ IsActiveMember : int  1 1 0 0 1 0 1 0 1 1 ...
 $ Exited         : int  1 0 1 0 0 1 0 1 0 0 ...


## Factorize an numerize features

In [5]:
df$Geography=as.numeric(factor(df$Geography))
df$Gender=as.numeric(factor(df$Gender))
str(df)

'data.frame':	10000 obs. of  11 variables:
 $ Geography      : num  1 3 1 1 3 3 1 2 1 1 ...
 $ Gender         : num  1 1 1 1 1 2 2 1 2 2 ...
 $ Age            : int  42 41 42 39 43 44 50 29 44 27 ...
 $ CreditScore    : int  619 608 502 699 850 645 822 376 501 684 ...
 $ EstimatedSalary: num  101349 112543 113932 93827 79084 ...
 $ Tenure         : int  2 1 8 1 2 8 7 4 4 2 ...
 $ Balance        : num  0 83808 159661 0 125511 ...
 $ NumOfProducts  : int  1 1 3 2 1 2 2 4 2 1 ...
 $ HasCrCard      : int  1 0 1 0 1 1 1 1 0 1 ...
 $ IsActiveMember : int  1 1 0 0 1 0 1 0 1 1 ...
 $ Exited         : int  1 0 1 0 0 1 0 1 0 0 ...


## Scale

In [6]:
# scale_features=c('Geography','Gender','Age','CreditScore','EstimatedSalary',
#                  'Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember')
# df[,scale_features]=scale(df[,scale_features])
# str(df)

## Train/test

In [7]:
set.seed(123)
split=sample.split(df$Exited, SplitRatio=0.8)
training_set=subset(df,split)
test_set=subset(df,!split)
writeLines(noquote(paste('Training set:',paste(dim(training_set),collapse='x'))))
writeLines(noquote(paste('Test set:',paste(dim(test_set),collapse='x'))))

Training set: 8000x11
Test set: 2000x11


## XGBoost

In [13]:
writeLines('\n=== XGBoost ===')
my_classifier=xgboost(data=as.matrix(training_set[,names(training_set)!='Exited']),
                      label=training_set$Exited,
                      nrounds=10)
writeLines('\nClassifier summary:')
print(summary(my_classifier))
prob_pred=predict(my_classifier,newdata=as.matrix(test_set[,names(test_set)!='Exited']))
y_pred=as.vector(prob_pred>0.5)
writeLines('\nConfusion matrix:')
print(table(test_set$Exited,y_pred))


=== XGBoost ===
[1]	train-rmse:0.419544 
[2]	train-rmse:0.372652 
[3]	train-rmse:0.344825 
[4]	train-rmse:0.329253 
[5]	train-rmse:0.318175 
[6]	train-rmse:0.311982 
[7]	train-rmse:0.308010 
[8]	train-rmse:0.305612 
[9]	train-rmse:0.303504 
[10]	train-rmse:0.301552 

Classifier summary:
               Length Class              Mode       
handle             1  xgb.Booster.handle externalptr
raw            36294  -none-             raw        
niter              1  -none-             numeric    
evaluation_log     2  data.table         list       
call              13  -none-             call       
params             1  -none-             list       
callbacks          3  -none-             list       

Confusion matrix:
   y_pred
    FALSE TRUE
  0  1530   63
  1   208  199


# With one hot encoding of Geography

## Read data

In [14]:
root_dir="~/Documents/Education/20170623 Udemy - Machine Learning A-Z: Hands-On Python and R in Data Science/"
work_dir="Course data/Part 10 - Model Selection & Boosting/Section 49 - XGBoost"
setwd(paste(root_dir,work_dir,sep=''))
df=read.csv('Churn_Modelling.csv')[c('Geography','Gender','Age','CreditScore','EstimatedSalary',
                                     'Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember',
                                     'Exited')]
str(df)

'data.frame':	10000 obs. of  11 variables:
 $ Geography      : Factor w/ 3 levels "France","Germany",..: 1 3 1 1 3 3 1 2 1 1 ...
 $ Gender         : Factor w/ 2 levels "Female","Male": 1 1 1 1 1 2 2 1 2 2 ...
 $ Age            : int  42 41 42 39 43 44 50 29 44 27 ...
 $ CreditScore    : int  619 608 502 699 850 645 822 376 501 684 ...
 $ EstimatedSalary: num  101349 112543 113932 93827 79084 ...
 $ Tenure         : int  2 1 8 1 2 8 7 4 4 2 ...
 $ Balance        : num  0 83808 159661 0 125511 ...
 $ NumOfProducts  : int  1 1 3 2 1 2 2 4 2 1 ...
 $ HasCrCard      : int  1 0 1 0 1 1 1 1 0 1 ...
 $ IsActiveMember : int  1 1 0 0 1 0 1 0 1 1 ...
 $ Exited         : int  1 0 1 0 0 1 0 1 0 0 ...


## Factorize an numerize features

In [15]:
df=cbind(data.frame(model.matrix(~Geography-1,df))[-1],df[,!(colnames(df)=='Geography')])
df$Gender=as.numeric(factor(df$Gender))
str(df)

'data.frame':	10000 obs. of  12 variables:
 $ GeographyGermany: num  0 0 0 0 0 0 0 1 0 0 ...
 $ GeographySpain  : num  0 1 0 0 1 1 0 0 0 0 ...
 $ Gender          : num  1 1 1 1 1 2 2 1 2 2 ...
 $ Age             : int  42 41 42 39 43 44 50 29 44 27 ...
 $ CreditScore     : int  619 608 502 699 850 645 822 376 501 684 ...
 $ EstimatedSalary : num  101349 112543 113932 93827 79084 ...
 $ Tenure          : int  2 1 8 1 2 8 7 4 4 2 ...
 $ Balance         : num  0 83808 159661 0 125511 ...
 $ NumOfProducts   : int  1 1 3 2 1 2 2 4 2 1 ...
 $ HasCrCard       : int  1 0 1 0 1 1 1 1 0 1 ...
 $ IsActiveMember  : int  1 1 0 0 1 0 1 0 1 1 ...
 $ Exited          : int  1 0 1 0 0 1 0 1 0 0 ...


## Scale

In [16]:
# scale_features=c('GeographyGermany','GeographySpain','Gender','Age','CreditScore','EstimatedSalary',
#                  'Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember')
# df[,scale_features]=scale(df[,scale_features])
# str(df)

## Train/test

In [17]:
set.seed(123)
split=sample.split(df$Exited, SplitRatio=0.8)
training_set=subset(df,split)
test_set=subset(df,!split)
writeLines(noquote(paste('Training set:',paste(dim(training_set),collapse='x'))))
writeLines(noquote(paste('Test set:',paste(dim(test_set),collapse='x'))))

Training set: 8000x12
Test set: 2000x12


## XGBoost

In [18]:
writeLines('\n=== XGBoost ===')
my_classifier=xgboost(data=as.matrix(training_set[,names(training_set)!='Exited']),
                      label=training_set$Exited,
                      nrounds=10)
writeLines('\nClassifier summary:')
print(summary(my_classifier))
prob_pred=predict(my_classifier,newdata=as.matrix(test_set[,names(test_set)!='Exited']))
y_pred=as.vector(prob_pred>0.5)
writeLines('\nConfusion matrix:')
print(table(test_set$Exited,y_pred))


=== XGBoost ===
[1]	train-rmse:0.417724 
[2]	train-rmse:0.369587 
[3]	train-rmse:0.342100 
[4]	train-rmse:0.325539 
[5]	train-rmse:0.315935 
[6]	train-rmse:0.310204 
[7]	train-rmse:0.305209 
[8]	train-rmse:0.302848 
[9]	train-rmse:0.300243 
[10]	train-rmse:0.296988 

Classifier summary:
               Length Class              Mode       
handle             1  xgb.Booster.handle externalptr
raw            36942  -none-             raw        
niter              1  -none-             numeric    
evaluation_log     2  data.table         list       
call              13  -none-             call       
params             1  -none-             list       
callbacks          3  -none-             list       

Confusion matrix:
   y_pred
    FALSE TRUE
  0  1539   54
  1   209  198
